In [8]:
import sys
import xarray as xr
sys.path.append(r'C:\Users\everett\Documents\GitHub\camus_to')



data_file = r"C:\Users\everett\Documents\GitHub\camus_to\data\clean\camus_to.nc"

with xr.open_dataset(r"C:\Users\everett\Documents\GitHub\camus_to\data\clean\camus_to.nc") as ds:
    df = ds.sel(station="HY006").to_dataframe().drop(columns=["station"]).copy()


df

,discharge,stage,precipitation,precipitation_ec,temperature,rh,pressure
time,,,,,,,
2000-01-01 00:00:00,NaN,NaN,NaN,NaN,1.050000,70.500000,100.385000
2000-01-01 00:15:00,NaN,NaN,NaN,NaN,1.050000,70.875000,100.375000
2000-01-01 00:30:00,NaN,NaN,NaN,NaN,1.050000,71.250000,100.365000
2000-01-01 00:45:00,NaN,NaN,NaN,NaN,1.050000,71.625000,100.355000
2000-01-01 01:00:00,NaN,NaN,NaN,NaN,1.050000,72.000000,100.345000
...,...,...,...,...,...,...,...
2024-06-23 23:00:00,NaN,NaN,NaN,0.0,19.100000,67.666667,98.940000
2024-06-23 23:15:00,NaN,NaN,NaN,0.0,19.041667,67.666667,98.953333
2024-06-23 23:30:00,NaN,NaN,NaN,0.0,18.983333,67.666667,98.966667
